In [1]:
import numpy as np
from scipy.io import loadmat
import pandas as pd
from scipy import sparse
import subprocess
import re
import os

In [2]:
 # Doc-term
doc_term_counts = sparse.load_npz('../../../../data/PubMed5/PubMed5_docWordMat.npz')
# Labels
labels_df = pd.read_csv('../../../../data/PubMed5/PubMed5_label.csv', 
                     usecols = [0], delim_whitespace = True)
labels = labels_df.values.flatten()
# Terms
terms_df = pd.read_csv('../../../../data/PubMed5/PubMed5_wordList.csv', 
                     usecols = [0], delim_whitespace = True)
terms = terms_df.values.flatten()
# Permutation
tmp_perm = np.random.RandomState(seed=42).permutation(doc_term_counts.shape[0])
#np.take(doc_term_counts, tmp_perm,axis=0,out = doc_term_counts)
doc_term_counts = doc_term_counts[tmp_perm,:]
labels = [labels[i] for i in tmp_perm.tolist()]

NPMI Within

In [3]:
n_top_terms=30

In [15]:

# PUBMED5 top terms
word_list = ['wa', 'macular', 'eye', 'visual', 'amd', 'retinal', 'acuity', 'associate', 'degeneration', 'group']
word_list = ['otitis', 'ear', 'child', 'media', 'middle', 'acute', 'antibiotics', 'aom', 'tube', 'om']
word_list = ['stone', 'renal', 'calcium', 'urinary', 'kidney', 'oxalate', 'rate', 'urine', 'percutaneous', 'calculi']
word_list = ['allergy', 'nasal', 'rhinitis', 'symptom', 'pollen', 'seasonal', 'effect', 'allergens', 'asthma', 'increase']
word_list = ['migraine', 'patient', 'headache', 'study', 'thi', 'treatment', 'ar', 'pain', 'attack', 'associate']

# AMD vs OTITIS
word_list = ["wa", "macular", "eye", "visual", "amd", "retinal", "acuity", "associate", "degeneration", 
             "group", "neovascular", "significant", "month", "diabetic", "no", "improve", "risk", 
             "choroiditis", "edema", "measure", "intravitreal", "result", "thick", "change", "injection", 
             "case", "evaluate", "development", "show", "factor"]
word_list = ["otitis","ear","children","media","middle","acute","antibiotics","aom","tube","om","ag",
             "hear","effusion","year","chronic","dai","infection","present","cause","isolate",
             "recurrent","pneumonia","bacterial","found","tympan","vaccin","mastoiditis","complication",
             "pneumococcal","influenza"]
word_list = ["stone","renal","calcium","urinary","kidney","oxalate","rate","urine", "percutaneous",
             "calculi","crystal","lower","shock","lithotripsy","wave", "fragment","ureter","acid",
             "require","excretion","nephrolithotomy","procedure", "formation","protein","material",
             "tract","management","metabolite","uric","pcnl"]
word_list = ["allergic", "nasal", "rhinitis", "symptom", "pollen", "seasonal", "effect", "allergen", 
             "asthma", "increase", "significantly", "subject", "score", "level", "cell", "ig", 
             "placebo", "allergy", "test", "sensitive", "specific", "total", "immunotherapy", 
             "active", "grass", "eosinophil", "intranasal", "reduce", "challenge", "daily"]
word_list = ["migraine", "patient", "headache", "study", "thi", "treatment", "ar", "pain", "clinical", 
             "compare", "attack", "than", "may", "different", "dure", "aura", "report", "control", 
             "severe", "ha", "mg", "medical", "response", "efficacy", "sumatriptan", "assess", 
             "include", "data", "triptan", "drug"]

word_list = ["otitis","ear","children","media","middle","acute","antibiotics","aom","tube","om","ag",
             "hear","effusion","year","chronic","dai","infection","present","cause","isolate",
             "recurrent","pneumonia","bacterial","found","tympan","vaccin","mastoiditis","complication",
             "pneumococcal","influenza"]


n_words = len(word_list)
n_pairs = int((n_words*(n_words-1)/2))

all_npmi = np.zeros((n_words, n_words))

topic_file = '../../../../../data/Palmetto/topic_files/current_topics.txt'
out_file = '../../../../../data/Palmetto/topic_files/current_topics_out.txt'

In [16]:

outF = open(topic_file, "w")
for wi in range(n_words-1):
    # Write all word pairs in a topic file
    for wj in range(wi+1, n_words):
        line = '{} {}\n'.format(word_list[wi], word_list[wj])
        outF.write(line)
outF.close()  

In [45]:
s = '0,80604'
s=s.replace(',','.')
print(s)

0.80604


In [18]:
file1 = open(out_file, 'r') 
#next(file1)
Lines = file1.readlines()[1:]

# Strips the newline character
for line in Lines: 
    tmp_list = line.strip().split()
    tmp_value = tmp_list[1]
    #print(tmp_value)
    tmp_value=tmp_value.replace(',','.')
    #print(tmp_value)
    #print(type(tmp_value))
    tmp_value=float(tmp_value)
    #print(type(tmp_value))
    tmp_wi_str = (tmp_list[2])[1:-1]
    #print(tmp_wi_str)
    tmp_wj_str = (tmp_list[3])[0:-1]
    tmp_wi_idx = word_list.index(tmp_wi_str)
    tmp_wj_idx = word_list.index(tmp_wj_str)
    all_npmi[tmp_wi_idx, tmp_wj_idx] = tmp_value
    all_npmi[tmp_wj_idx, tmp_wi_idx] = all_npmi[tmp_wi_idx, tmp_wj_idx]
    #print(tmp_wi_str, tmp_wj_str, tmp_value)

In [19]:
#print(all_npmi)
# Compute the NPMI3 scores
k = 3
all_npmi_argsort = np.argsort(-all_npmi, axis = 0)
all_npmi_score = []

for i_word in range(len(word_list)):
    tmp_order_idx = all_npmi_argsort[:,i_word].flatten().tolist()
    tmp_order_idx = [idx for idx in tmp_order_idx if idx != i_word ]
    tmp_npmi_score = all_npmi[tmp_order_idx[:k], i_word]
    print(i_word, word_list[i_word], "--> ", [word_list[i] for i in tmp_order_idx[:k]], tmp_npmi_score)
    all_npmi_score.append(np.mean(tmp_npmi_score))

import pandas as pd

word_npmi_order_idx = np.argsort(-np.array(all_npmi_score))
word_npmi_order_str_series = pd.Series([word_list[i] for i in word_npmi_order_idx])
word_npmi_order_val_series = pd.Series([all_npmi_score[i] for i in word_npmi_order_idx])
pd_npmi=pd.concat([word_npmi_order_str_series, word_npmi_order_val_series], axis = 1)
pd_npmi.to_csv('../../../../../data/Palmetto/Npmi_for_graphs/npmi_otitis.csv',header=False,index=False)
pd_npmi


0 otitis -->  ['effusion', 'ear', 'acute'] [0.55789 0.48701 0.47456]
1 ear -->  ['otitis', 'infection', 'hear'] [0.48701 0.30489 0.28045]
2 children -->  ['found', 'ear', 'media'] [0.03622 0.02468 0.02054]
3 media -->  ['otitis', 'effusion', 'ag'] [0.44146 0.15831 0.10234]
4 middle -->  ['otitis', 'ear', 'effusion'] [0.22219 0.18929 0.12738]
5 acute -->  ['chronic', 'otitis', 'infection'] [0.50645 0.47456 0.36733]
6 antibiotics -->  ['bacterial', 'infection', 'otitis'] [0.46206 0.45937 0.39581]
7 aom -->  ['vaccin', 'tympan', 'mastoiditis'] [0.05674 0.02011 0.00537]
8 tube -->  ['otitis', 'ear', 'complication'] [0.26187 0.15994 0.08892]
9 om -->  ['hear', 'media', 'found'] [-0.01775 -0.02664 -0.04035]
10 ag -->  ['found', 'media', 'present'] [ 0.11957  0.10234 -0.02505]
11 hear -->  ['ear', 'otitis', 'acute'] [0.28045 0.21981 0.09826]
12 effusion -->  ['otitis', 'pneumonia', 'ear'] [0.55789 0.34456 0.27857]
13 year -->  ['pneumonia', 'pneumococcal', 'children'] [0.03708 0.01858 0.01836

,0,1
0,otitis,0.506487
1,infection,0.479157
2,pneumonia,0.465707
3,bacterial,0.463713
4,acute,0.449447
5,chronic,0.440783
6,antibiotics,0.439080
7,effusion,0.393673
8,recurrent,0.389943
9,complication,0.366057


In [20]:
words = [word_list[i] for i in word_npmi_order_idx]
matrix_pd=pd.DataFrame(all_npmi,index=words,columns=words)
matrix_pd

,otitis,infection,pneumonia,bacterial,acute,chronic,antibiotics,effusion,recurrent,complication,...,tympan,found,ag,present,mastoiditis,aom,children,year,dai,om
otitis,0.00000,0.48701,-0.28542,0.44146,0.22219,0.47456,0.39581,-0.06334,0.26187,-0.20776,...,0.43035,0.38669,0.36467,-0.34398,-0.00299,0.03365,-0.01772,-0.22582,-0.08150,-0.19186
infection,0.48701,0.00000,0.02468,0.01163,0.18929,0.14070,0.16808,-0.22721,0.15994,-0.37163,...,0.16180,0.09773,0.12970,-0.07480,-0.16686,-0.13022,-0.18159,0.09597,-0.24537,-0.35573
pneumonia,-0.28542,0.02468,0.00000,0.02054,-0.04180,0.01464,-0.37793,-0.26232,-0.05683,-0.40675,...,-0.37423,-0.39834,-0.40379,0.03622,-0.20197,-0.16534,-0.21671,-0.42480,-0.28048,-0.39084
bacterial,0.44146,0.01163,0.02054,0.00000,-0.00231,0.02749,-0.40251,-0.28690,-0.07883,-0.02664,...,0.03885,0.01338,-0.02939,0.04688,-0.22655,-0.18992,-0.24129,-0.04382,-0.30506,-0.41542
acute,0.22219,0.18929,-0.04180,-0.00231,0.00000,-0.02520,-0.41891,-0.30330,0.02159,-0.44773,...,0.00253,-0.43932,-0.03925,-0.01181,-0.24296,-0.20632,-0.25769,-0.05986,-0.32147,-0.43183
chronic,0.47456,0.14070,0.01464,0.02749,-0.02520,0.00000,0.24369,-0.20617,0.04605,-0.35060,...,0.27042,0.33006,0.29269,-0.07677,-0.14582,-0.10919,-0.16056,0.28213,-0.22433,0.22446
antibiotics,0.39581,0.16808,-0.37793,-0.40251,-0.41891,0.24369,0.00000,-0.15585,-0.35872,-0.30027,...,-0.26776,0.31480,0.46206,-0.43649,-0.09550,-0.05886,-0.11023,0.23454,-0.17401,-0.28437
effusion,-0.06334,-0.22721,-0.26232,-0.28690,-0.30330,-0.20617,-0.15585,0.00000,-0.24311,-0.18467,...,-0.15215,-0.17626,-0.18171,-0.32089,0.02011,0.05674,0.00537,-0.20272,-0.05840,-0.16876
recurrent,0.26187,0.15994,-0.05683,-0.07883,0.02159,0.04605,-0.35872,-0.24311,0.00000,-0.38753,...,-0.35502,0.06769,-0.38458,-0.10841,-0.18276,-0.14612,-0.19750,0.08892,-0.26127,-0.37163
complication,-0.20776,-0.37163,-0.40675,-0.02664,-0.44773,-0.35060,-0.30027,-0.18467,-0.38753,0.00000,...,-0.29658,-0.32068,-0.32614,-0.04035,-0.12432,-0.08768,-0.13905,-0.34715,-0.20283,-0.31319


In [21]:
# Save all NPMI values as matrix
matrix_pd.to_csv("../../../../../data/PubMed5/NPMI/allNPMI_30w_k3_otitis.csv")

In [12]:
# Compute all pairwise average
all_npmi_mean = (np.triu(all_npmi, 1).sum())/(all_npmi.shape[0]*(all_npmi.shape[0]-1)/2)
print("# NPMI average: ", all_npmi_mean)

# NPMI average:  -0.08272436781609196


NPMI Between

In [14]:
# PUBMED5 frequency top terms
## -- 10 -- ##
#word_list_amd = ['wa', 'macular', 'eye', 'visual', 'amd', 'retinal', 'acuity', 'associate', 'degeneration', 'group']
##word_list_otitis = ['otitis', 'ear', 'children', 'media', 'middle', 'acute', 'antibiotics', 'aom', 'tube', 'om']
#word_list_otitis = ['otitis', 'ear', 'child', 'media', 'middle', 'acute', 'antibiotics', 'aom', 'tube', 'om']
#word_list_kidney = ['stone', 'renal', 'calcium', 'urinary', 'kidney', 'oxalate', 'rate', 'urine', 'percutaneous', 'calculi']
#word_list_hayfever = ['allergy', 'nasal', 'rhinitis', 'symptom', 'pollen', 'seasonal', 'effect', 'allergens', 'asthma', 'increase']
#word_list_migraine = ['migraine', 'patient', 'headache', 'study', 'thi', 'treatment', 'ar', 'pain', 'attack', 'associate']

# PUBMED5 NPMIk top terms
## -- 30 -- ##
word_list_amd = ["macular", "degeneration", "retinal", "edema", "diabetic", "acuity", "visual", "amd", 
                  "injection", "eye", "neovascular", "risk", "factor", "intravitreal", "significant", 
                  "measure", "associate", "evaluate", "improve", "result", "change", "development", 
                  "case", "thick", "show", "choroiditis", "group", "month", "no", "wa"]

word_list_otitis = ["otitis", "infection", "pneumonia", "bacterial", "acute", "chronic", "antibiotics", 
                    "recurrent", "effusion", "complication", "influenza", "ear", "cause", "pneumococcal", 
                    "isolate", "media", "tube", "hear", "middle", "vaccin", "present", "found", "tympan", 
                    "mastoiditis", "children", "ag", "year", "aom", "dai", "om"]

word_list_kidney = ["urinary", "excretion", "ureter", "urine", "kidney", "renal", "uric", "oxalate", 
                    "acid", "metabolite", "calcium", "lithotripsy", "tract", "calculi", "protein", 
                    "crystal", "stone", "percutaneous", "procedure", "shock", "wave", "lower", 
                    "formation", "nephrolithotomy", "rate", "fragment", "require", "material", 
                    "management", "pcnl"]

word_list_hayfever = ["allergic", "rhinitis", "allergy", "asthma", "allergen", "immunotherapy", "pollen", "nasal", 
                      "symptom", "eosinophil", "cell", "seasonal", "effect", "sensitive", "placebo", "reduce", 
                      "increase", "test", "specific", "grass", "level", "subject", "total", "active", "daily", 
                      "intranasal", "score", "challenge", "ig", "significantly"]

word_list_otitis = ["otitis", "infection", "pneumonia", "bacterial", "acute", "chronic", "antibiotics", 
                    "recurrent", "effusion", "complication", "influenza", "ear", "cause", "pneumococcal", 
                    "isolate", "media", "tube", "hear", "middle", "vaccin", "present", "found", "tympan", 
                    "mastoiditis", "children", "ag", "year", "aom", "dai", "om"]

In [15]:
# One vs Others,
word_list = word_list_otitis
word_list.extend(word_list_hayfever)

n_words = len(word_list)
n_pairs = int((n_words*(n_words-1)/2))

all_npmi = np.zeros((n_words, n_words))

outF = open(topic_file, "w")
for wi in range(n_words-1):
    #print("# word", wi)
    # Write all word pairs in a topic file
    
    for wj in range(wi+1, n_words):
        line = '{} {}\n'.format(word_list[wi], word_list[wj])
        outF.write(line)
outF.close()


In [19]:

# Read out file line by line
file1 = open(out_file, 'r') 
#next(file1)
Lines = file1.readlines()[1:]

# Strips the newline character
for line in Lines: 
    tmp_list = line.strip().split()
    #print(tmp_list)
    tmp_value = tmp_list[1]
    tmp_value=tmp_value.replace(',','.')
    tmp_wi_str = (tmp_list[2])[1:-1]
    tmp_wj_str = (tmp_list[3])[0:-1]
    tmp_wi_idx = word_list.index(tmp_wi_str)
    tmp_wj_idx = word_list.index(tmp_wj_str)
    all_npmi[tmp_wi_idx, tmp_wj_idx] = tmp_value
    all_npmi[tmp_wj_idx, tmp_wi_idx] = all_npmi[tmp_wi_idx, tmp_wj_idx]
    #print(tmp_wi_str, tmp_wj_str, tmp_value)


60


In [21]:
#print(all_npmi)
# Compute the NPMIk scores
k = 3
#print(word_list)
sub_npmi = all_npmi[n_top_terms:(2*n_top_terms),:n_top_terms]
sub_npmi_argsort = np.argsort(-sub_npmi, axis = 0)
sub_npmi_score = []

for i_word in range(sub_npmi.shape[1]):
    tmp_order_idx = sub_npmi_argsort[:,i_word].flatten().tolist()
    tmp_npmi_score = sub_npmi[tmp_order_idx[:k], i_word]
    print(i_word, word_list[i_word], "--> ", [word_list[i+n_top_terms] for i in tmp_order_idx[:k]], 
          tmp_npmi_score)
    sub_npmi_score.append(np.mean(tmp_npmi_score))

import pandas as pd
word_npmi_str_series = pd.Series(word_list[:n_top_terms])
word_npmi_val_series = pd.Series(sub_npmi_score)
pd.concat([word_npmi_str_series, word_npmi_val_series], axis = 1)

30
0 otitis -->  ['symptom', 'significantly', 'intranasal'] [ 0.2691  0.     -0.065 ]
1 infection -->  ['symptom', 'rhinitis', 'allergic'] [0.36337 0.31269 0.2861 ]
2 pneumonia -->  ['asthma', 'symptom', 'cell'] [0.32708 0.22523 0.05545]
3 bacterial -->  ['cell', 'allergy', 'symptom'] [0.34949 0.21211 0.20754]
4 acute -->  ['symptom', 'asthma', 'allergic'] [0.35116 0.3237  0.2444 ]
5 chronic -->  ['asthma', 'rhinitis', 'symptom'] [0.43612 0.40047 0.33726]
6 antibiotics -->  ['allergic', 'allergy', 'symptom'] [0.30648 0.25557 0.23859]
7 recurrent -->  ['symptom', 'asthma', 'subject'] [0.26746 0.24168 0.09593]
8 effusion -->  ['symptom', 'cell', 'significantly'] [0.24015 0.16054 0.     ]
9 complication -->  ['asthma', 'symptom', 'allergic'] [0.24323 0.23471 0.19441]
10 influenza -->  ['intranasal', 'seasonal', 'symptom'] [0.42585 0.33636 0.27252]
11 ear -->  ['nasal', 'sensitive', 'allergy'] [0.20378 0.19123 0.17231]
12 cause -->  ['symptom', 'allergic', 'allergy'] [0.2745  0.26602 0.202

,0,1
0,otitis,0.068033
1,infection,0.320720
2,pneumonia,0.202587
3,bacterial,0.256380
4,acute,0.306420
5,chronic,0.391283
6,antibiotics,0.266880
7,recurrent,0.201690
8,effusion,0.133563
9,complication,0.224117
